# Main Dashboard

In [9]:
# import libraries
from sklearn import metrics
from sklearn.model_selection import train_test_split
import lightgbm as ltb
import xgboost as xgb
import catboost as catb
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from explainerdashboard import ClassifierExplainer, ExplainerDashboard, ExplainerHub
from explainerdashboard.custom import *
import re

In [10]:
# import dataset
df = pd.read_csv("../ML/mental_health_data_cleaned.csv")

In [11]:
# correct column names
arp_data1 = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [12]:
# Create train and test datasets
y=arp_data1["treatment"]
X=arp_data1.drop("treatment", axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [13]:
################################# 1st DASHBOARD -- CATBOOST ####################################
###
###
# Create 1st Explainer Dashboard

model_catb_tuned = catb.CatBoostClassifier(n_estimators=50, max_depth=5)

#model_catb_tuned = catb.CatBoostClassifier(
#    depth=6, iterations=1000, learning_rate=0.1, min_data_in_leaf=399
#)

model_catb_tuned.fit(X_train, y_train)

explainer_catb = ClassifierExplainer(
                model_catb_tuned, X_test, y_test,
                labels=['Had Mental Illness', 'No Mental Ilness']
)

db_catb = ExplainerDashboard(explainer_catb, name='catb', title="CatBoost Explainer",
                    whatif=False,
                    shap_interaction=False,
                    decision_trees=False)

Learning rate set to 0.16566
0:	learn: 0.5633295	total: 5.67ms	remaining: 278ms
1:	learn: 0.4906695	total: 6.82ms	remaining: 164ms
2:	learn: 0.4249268	total: 8.05ms	remaining: 126ms
3:	learn: 0.3786904	total: 9.04ms	remaining: 104ms
4:	learn: 0.3559021	total: 10.1ms	remaining: 91ms
5:	learn: 0.3346699	total: 11ms	remaining: 80.6ms
6:	learn: 0.3165590	total: 11.9ms	remaining: 73.1ms
7:	learn: 0.3067550	total: 12.9ms	remaining: 67.6ms
8:	learn: 0.2969546	total: 13.7ms	remaining: 62.5ms
9:	learn: 0.2916692	total: 14.5ms	remaining: 58.1ms
10:	learn: 0.2836769	total: 16.1ms	remaining: 57.2ms
11:	learn: 0.2798688	total: 17.2ms	remaining: 54.3ms
12:	learn: 0.2776279	total: 18.2ms	remaining: 51.8ms
13:	learn: 0.2741613	total: 19.1ms	remaining: 49.1ms
14:	learn: 0.2708735	total: 20ms	remaining: 46.6ms
15:	learn: 0.2657027	total: 20.8ms	remaining: 44.1ms
16:	learn: 0.2627888	total: 21.5ms	remaining: 41.8ms
17:	learn: 0.2603269	total: 22.4ms	remaining: 39.8ms
18:	learn: 0.2556584	total: 23.2ms	re

In [14]:
################################# 2nd DASHBOARD -- XGBOOST ####################################
###
###
# Create 2nd Explainer Dashboard

model_xgb_tuned = xgb.XGBClassifier(n_estimators=50, max_depth=5)

model_xgb_tuned.fit(X_train, y_train)

explainer_xgb = ClassifierExplainer(
                model_xgb_tuned, X_test, y_test,
                labels=['Had Mental Illness', 'No Mental Ilness']
)

db_xgb = ExplainerDashboard(explainer_xgb, name='xgb', title="XGBoost Explainer",
                    whatif=False,
                    shap_interaction=False,
                    decision_trees=False)

Detected XGBClassifier model: Changing class type to XGBClassifierExplainer...
Note: model_output=='probability'. For XGBClassifier shap values normally get calculated against X_background, but paramater X_background=None, so using X instead
Generating self.shap_explainer = shap.TreeExplainer(model, X, model_output='probability', feature_perturbation='interventional')...
Note: Shap interaction values will not be available. If shap values in probability space are not necessary you can pass model_output='logodds' to get shap values in logodds without the need for a background dataset and also working shap interaction values...
Building ExplainerDashboard..
Detected notebook environment, consider setting mode='external', mode='inline' or mode='jupyterlab' to keep the notebook interactive while the dashboard is running...
Generating layout...
Calculating shap values...
Calculating prediction probabilities...
Calculating metrics...
Calculating confusion matrices...
Calculating classificatio

In [15]:
################################# 3rd DASHBOARD -- LIGHTGBM ####################################
###
###
# Create 3rd Explainer Dashboard

model_lgbm_tuned = ltb.LGBMClassifier(n_estimators=50, max_depth=5)

model_lgbm_tuned.fit(X_train, y_train)

explainer_lgbm = ClassifierExplainer(
                model_lgbm_tuned, X_test, y_test,
                labels=['Had Mental Illness', 'No Mental Ilness']
)

db_lgbm = ExplainerDashboard(explainer_lgbm, name='lgbm', title="LightGBM Explainer",
                    whatif=False,
                    shap_interaction=False,
                    decision_trees=False)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
Note: model_output=='probability'. For LGBMClassifier shap values normally get calculated against X_background, but paramater X_background=None, so using X instead
Generating self.shap_explainer = shap.TreeExplainer(model, X, model_output='probability', feature_perturbation='interventional')...
Note: Shap interaction values will not be available. If shap values in probability space are not necessary you can pass model_output='logodds' to get shap values in logodds without the need for a background dataset and also working shap interaction values...
Building ExplainerDashboard..
Detected notebook environment, consider setting mode='external', mode='inline' or mode='jupyterlab' to keep the notebook interactive while the dashboard is running...
Generating layout...
Calculating shap values...
Calculating prediction probabilities...
Calculating metrics...
Calcula

In [56]:
####################### Comparison DASHBOARD -- XGBOOST, LIGHTGBM, CATBOOST #####################
###
###
# Create a Comparison Dashboard
class ResultComparison(ExplainerComponent):
    def __init__(self, explainer_lgbm, explainer_xgb):
        super().__init__(explainer_catb)
                
        # ShapDependenceComponent
        self.confmat_catb_shap_dep = ShapDependenceComponent(explainer_catb, subtitle='CatBoost', cutoff=0.3,
                            hide_selector=True, hide_percentage=True)
        self.confmat_xgb_shap_dep = ShapDependenceComponent(explainer_xgb, subtitle='XGBoost', cutoff=0.3,
                            hide_selector=True, hide_percentage=True)
        
        # PrecisionComponent
        self.confmat_catb_prec = PrecisionComponent(explainer_catb, subtitle='CatBoost', cutoff=0.3,
                            hide_selector=True, hide_percentage=True)
        self.confmat_xgb_prec = PrecisionComponent(explainer_xgb, subtitle='XGBoost', cutoff=0.3,
                            hide_selector=True, hide_percentage=True)
       
    def layout(self):
        return dbc.Container([
            dbc.Row([
                dbc.Col([
                    self.confmat_catb_shap_dep.layout()
                ]),
                dbc.Col([
                    self.confmat_xgb_shap_dep.layout()
                ]),
            ]),
            dbc.Row([
                dbc.Col([
                    self.confmat_catb_prec.layout()
                ]),
                dbc.Col([
                    self.confmat_xgb_prec.layout()
                ]),

            ]),
        ])
    
tab = ResultComparison(explainer_catb, explainer_xgb)

db_comp = ExplainerDashboard(explainer_xgb, tabs=tab, name='comparison', title="Comarison Results",
                            decision_trees=False,
                            whatif=False,
                            shap_interaction=False,
                            contributions=False, 
                             model_summary=False, 
                             shap_dependence=False)
# run comparison dashboard alone
#db_comp.run(8052)


Building ExplainerDashboard..
Detected notebook environment, consider setting mode='external', mode='inline' or mode='jupyterlab' to keep the notebook interactive while the dashboard is running...
Generating layout...
Calculating dependencies...
Reminder: you can store the explainer (including calculated dependencies) with explainer.dump('explainer.joblib') and reload with e.g. ClassifierExplainer.from_file('explainer.joblib')
Registering callbacks...
Starting ExplainerDashboard on http://192.168.178.29:8052
Dash is running on http://0.0.0.0:8052/

Dash is running on http://0.0.0.0:8052/

Dash is running on http://0.0.0.0:8052/

Dash is running on http://0.0.0.0:8052/

Dash is running on http://0.0.0.0:8052/

Dash is running on http://0.0.0.0:8052/

Dash is running on http://0.0.0.0:8052/

Dash is running on http://0.0.0.0:8052/

Dash is running on http://0.0.0.0:8052/

Dash is running on http://0.0.0.0:8052/

Dash is running on http://0.0.0.0:8052/

Dash is running on http://0.0.0.0:8

 * Running on http://0.0.0.0:8052/ (Press CTRL+C to quit)
127.0.0.1 - - [19/May/2022 09:58:43] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2022 09:58:44] "GET /assets/bootstrap.min.css?m=1651904588.9278898 HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2022 09:58:44] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_3_1m1651157205.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2022 09:58:44] "GET /_dash-component-suites/dash/deps/react@16.v2_3_1m1651157205.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2022 09:58:44] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_3_1m1651157205.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2022 09:58:44] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_3_1m1651157205.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2022 09:58:44] "GET /_dash-component-suites/dash_bootstrap_components/_components/dash_bootstrap_components.v0_13_1m1651904588.min.js HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2022 09:58:44] "GET /_dash-component-suites/dash/

In [16]:
# Run all dashboards in a single host
#db_catb, db_lgbm, db_xgb, 
hub = ExplainerHub([db_xgb, db_lgbm, db_catb])
hub.run(8053)

Using random SECRET_KEY: 6ef1ff50-edfb-488a-90fb-fec1fe21a2c1, please set it on your app.config["SECRET_KEY"]


Building ExplainerDashboard..
Detected notebook environment, consider setting mode='external', mode='inline' or mode='jupyterlab' to keep the notebook interactive while the dashboard is running...
Generating layout...
Calculating dependencies...
Reminder: you can store the explainer (including calculated dependencies) with explainer.dump('explainer.joblib') and reload with e.g. ClassifierExplainer.from_file('explainer.joblib')
Registering callbacks...
Building ExplainerDashboard..
Detected notebook environment, consider setting mode='external', mode='inline' or mode='jupyterlab' to keep the notebook interactive while the dashboard is running...
Generating layout...
Calculating dependencies...
Reminder: you can store the explainer (including calculated dependencies) with explainer.dump('explainer.joblib') and reload with e.g. ClassifierExplainer.from_file('explainer.joblib')
Registering callbacks...
Building ExplainerDashboard..
Detected notebook environment, consider setting mode='exte

 * Running on http://0.0.0.0:8053/ (Press CTRL+C to quit)
127.0.0.1 - - [29/May/2022 11:21:19] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2022 11:21:19] "GET /static/bootstrap.min.css HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2022 11:21:19] "GET /static/jquery-3.5.1.slim.min.js HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2022 11:21:19] "GET /static/bootstrap.min.js HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2022 11:21:21] "GET /index/ HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2022 11:21:21] "GET /index/assets/bootstrap.min.css?m=1651904588.9278898 HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2022 11:21:21] "GET /index/_dash-component-suites/dash/deps/polyfill@7.v2_3_1m1651157205.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2022 11:21:21] "GET /index/_dash-component-suites/dash_bootstrap_components/_components/dash_bootstrap_components.v0_13_1m1651904588.min.js HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2022 11:21:21] "GET /index/_dash-component-suites/dash/deps/react-dom@16.v2_3_1m1651157205.14.0.min.js HTTP/